# Gestione delle date in pandas 

Questo notebook esplora come lavorare con le date e gli orari nei dati tabulari usando pandas.  

Immagina di avere misurazioni di potenza elettrica o temperatura raccolte ogni 15 minuti: spesso i timestamp arrivano come testo semplice, e per analizzarli correttamente devi convertirli in un formato che pandas possa gestire.  
Questo permette di ordinare i dati nel tempo, identificare buchi nelle serie, aggregare valori per ore o giorni, creare previsioni basate su ritardi o medie mobili, e persino gestire fusi orari senza errori.  

In pratica, è essenziale per chiunque analizzi dati che cambiano nel tempo, come in energia, meteorologia o finanza, per evitare confusioni e ottenere risultati affidabili.  




<a id="indice"></a>
## Indice

- [Dataset toy](#dataset-toy)
- [`datetime` in pandas](#datetime)
- [Formati: il minimo indispensabile](#formati)
- [Il tempo come indice](#tempo-indice)
- [Attributi temporali](#attributi)
- [Frequenza](#frequenza)
- [Missing che emergono con l’allineamento](#missing)
- [Resampling](#resampling)
- [Shift e rolling](#shift-rolling)
- [Differenze tra date](#differenze)
- [Fuso orario e ora legale](#timezone)


<a id="dataset-toy"></a>
## Dataset toy

Costruiamo un dataset piccolo ma realistico: misure quartorarie di potenza (`power_kw`) e temperatura (`temp_c`).  
Il timestamp è intenzionalmente una **stringa** in formato italiano (`DD/MM/YYYY HH:MM`).  
Poi inseriamo qualche buco per simulare acquisizioni mancanti.


In [ ]:
import numpy as np
np.random.seed(42)
import pandas as pd

pd.set_option("display.max_rows", 10)
pd.set_option("display.max_columns", 20)

rng = pd.date_range("2025-03-01 00:00", periods=3 * 24 * 4, freq="15min")

# pattern giornaliero semplice + rumore: giusto per avere una serie “viva”
hours = rng.hour + rng.minute / 60
power_kw = 220 + 60 * np.sin(2 * np.pi * (hours / 24)) + np.random.normal(0, 8, size=len(rng))
temp_c = 12 + 5 * np.sin(2 * np.pi * ((hours - 6) / 24)) + np.random.normal(0, 0.7, size=len(rng))

df = pd.DataFrame({
    "timestamp_str": rng.strftime("%d/%m/%Y %H:%M"),  # formato tipico in Italia
    "power_kw": power_kw.round(1),
    "temp_c": temp_c.round(1),
})

# rimuoviamo alcune righe per simulare missing (buchi nella serie)
drop_idx = np.random.choice(df.index, size=18, replace=False)  # ~2.5% su 288 punti
df = df.drop(drop_idx).reset_index(drop=True)

df.head()


<div style="background-color:#f5f5f5; padding:12px 14px; border-left:4px solid #5cb85c; border-radius:2px;">

**Esercizio**

Stampa `df.dtypes` e verifica che `timestamp_str` risulti `object` (testo).

</div>

In [ ]:
# Stampa df.dtypes
...

<a id="datetime"></a>
## `datetime` in pandas

Una colonna di testo che “sembra una data” è comunque testo semplice, e pandas non può sfruttarne le proprietà temporali senza conversione.  
`pd.to_datetime` trasforma una colonna in un tipo temporale gestibile (`datetime64[ns]`), abilitando operazioni come ordinamento, filtraggio per periodi e calcolo di intervalli.  

Immagina i tuoi timestamp di potenza elettrica o temperatura: arrivano spesso come stringhe (es. "01/03/2025 00:00"), ma convertirli permette di riconoscere sequenze irregolari, aggregare per ore o giorni, e persino prevedere trend senza errori di interpretazione.  
In pratica, è il primo passo per dati temporali affidabili in energia, meteorologia o finanza, evitando confusioni che potrebbero invalidare le analisi.

Documentazione:
- `pd.to_datetime`: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html


In [ ]:
df["timestamp"] = pd.to_datetime(df["timestamp_str"], dayfirst=True)
df[["timestamp_str", "timestamp"]].head()

In [ ]:
df.dtypes

<div style="background-color:#f5f5f5; padding:12px 14px; border-left:4px solid #999; border-radius:2px;">

**Nota importante**

In Italia il caso più comune è `giorno/mese/anno`, quindi `dayfirst=True` è spesso la scelta corretta. 

Quando il formato è noto e stabile, usare `format=...` rende la conversione più robusta.

</div>

<div style="background-color:#f5f5f5; padding:12px 14px; border-left:4px solid #5cb85c; border-radius:2px;">

**Esercizio**

Crea una nuova colonna `timestamp_2` convertendo di nuovo `timestamp_str`, ma questa volta usando anche `format="%d/%m/%Y %H:%M"`. Confronta le prime 5 righe con `timestamp`.

</div>

In [ ]:
#df["timestamp_2"] = pd.to_datetime(df["timestamp_str"], format=...)
#df[["timestamp", "timestamp_2"]].head()

<a id="formati"></a>
## Formati: il minimo indispensabile

Il formato serve soprattutto quando la stringa è **ambigua** o non standard.  
Esempio classico: `01/02/2025` può essere 1 febbraio oppure 2 gennaio (dipende dal contesto).

Documentazione:
- opzioni di `to_datetime` (tra cui `format`, `dayfirst`, `errors`): https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html


In [ ]:
# Esempio ambiguità giorno/mese + gestione errori
ambigue = pd.Series(["01/02/2025 08:00", "13/02/2025 08:00"])

parsed_dayfirst = pd.to_datetime(ambigue, dayfirst=True)
parsed_monthfirst = pd.to_datetime(ambigue, dayfirst=False, errors="coerce")  # qui un valore diventa NaT

pd.DataFrame({
    "stringa": ambigue,
    "dayfirst=True": parsed_dayfirst,
    "dayfirst=False (coerce)": parsed_monthfirst,
})

<div style="background-color:#f5f5f5; padding:12px 14px; border-left:4px solid #5cb85c; border-radius:2px;">

**Esercizio**

Sostituisci `errors="coerce"` con `errors="raise"` (o rimuovi proprio `errors`) e osserva cosa succede.

</div>

In [ ]:
# Prova a gestire l'errore con 'raise' (opzione di errors)
#parsed_monthfirst_raise = pd.to_datetime(ambigue, dayfirst=False, errors=...)
#parsed_monthfirst_raise

<a id="tempo-indice"></a>
## Il tempo come indice

Quando il timestamp è l’indice, pandas abilita selezioni “per periodo” senza calcoli manuali.  
Prima di tutto: ordinare l’indice. Con dati reali capita spesso di avere righe fuori ordine.

Documentazione:
- `DataFrame.set_index`: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.set_index.html
- indicizzazione temporale (user guide): https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html


In [ ]:
df_ts = (
    df[["timestamp", "power_kw", "temp_c"]]
    .set_index("timestamp")
    .sort_index()
)

df_ts.head()

In [ ]:
# slicing per periodo (anno/mese/giorno)
df_ts.loc["2025-03-02"].head()

<div style="background-color:#f5f5f5; padding:12px 14px; border-left:4px solid #5cb85c; border-radius:2px;">

**Esercizio**

Seleziona il periodo dal `2025-03-01 12:00` al `2025-03-01 18:00` usando `.loc[...]`.

</div>

In [ ]:
df_ts...

<a id="attributi"></a>
## Attributi temporali

Una volta che il tempo è `datetime`, estrarre componenti temporali diventa semplice (mese, ora, giorno della settimana).  
Se il tempo è indice, si lavora via `df.index`; se è una colonna, si usa `.dt`.

Documentazione:
- `.dt` accessor: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html
- attributi del `DatetimeIndex`: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DatetimeIndex.html


In [ ]:
df_feat = df_ts.copy()
df_feat["hour"] = df_feat.index.hour
df_feat["dayofweek"] = df_feat.index.dayofweek  # 0=lunedì
df_feat["month"] = df_feat.index.month

df_feat.head()

<div style="background-color:#f5f5f5; padding:12px 14px; border-left:4px solid #5cb85c; border-radius:2px;">

**Esercizio**

Aggiungi le colonne `date` (solo data) e `is_weekend` (sab/dom). 

Suggerimento: `df.index.date` e `dayofweek`.

</div>

In [ ]:
df_feat_ex = df_feat.copy()
# Extract the date part from the index
df_feat_ex["date"] = ...

In [ ]:
# Extract if the day is weekend (Saturday=5, Sunday=6), dayofweek starts from 0=Monday
df_feat_ex["is_weekend"] = ...

In [ ]:
# final display
df_feat_ex[["power_kw", "hour", "dayofweek", "date", "is_weekend"]].head()

<a id="frequenza"></a>
## Frequenza

“Frequenza” significa intervallo atteso tra due timestamp consecutivi (15 minuti, 1 ora, 1 giorno…).  
Con dati puliti e regolari, pandas può inferirla; con buchi o irregolarità spesso no.

Frequenze comuni (utility / reporting):
- `15min` (quartorario), `h` (orario), `D` (giornaliero), `MS` (inizio mese)

Documentazione:
- `DatetimeIndex.inferred_freq`: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DatetimeIndex.inferred_freq.html
- alias delle frequenze (offset): https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects


In [ ]:
df_ts.index.inferred_freq

Non restituisce niente perché non è riusito ad inferire una frequenza

<div style="background-color:#f5f5f5; padding:12px 14px; border-left:4px solid #5cb85c; border-radius:2px;">

**Esercizio**

Stampa la differenza tra due timestamp consecutivi usando `df_ts.index.to_series().diff().value_counts().head()`.

</div>

In [ ]:
df_ts.index.to_series().diff().value_counts().head()

<a id="missing"></a>
## Missing che emergono con l’allineamento

Quando “forzi” una frequenza regolare, i timestamp mancanti diventano righe con `NaN`.  
Questo è utile: rende visibili buchi che altrimenti restano nascosti.

Qui usiamo `asfreq` per allineare a frequenza quartoraria (`15min`).

Documentazione:
- `DataFrame.asfreq`: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.asfreq.html
- `isna`: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.isna.html


In [ ]:
df_qh = df_ts.asfreq("15min")
df_qh

In [ ]:
missing_counts = df_qh.isna().sum()
missing_counts

In [ ]:
# dove mancano i valori di power_kw?
missing_timestamps = df_qh.index[df_qh["power_kw"].isna()]
missing_timestamps[:10]

In [ ]:
# visualizzazione con Plotly dei dati prima e dopo asfreq (usa pallini oltre alla traccia)
import plotly.express as px

fig = px.line(df_ts, x=df_ts.index, y="power_kw", title="Dati originali (con buchi)")
fig.show()
fig = px.line(df_qh, x=df_qh.index, y="power_kw", title="Dati con asfreq (buchi evidenziati)")
fig.show()


Ora che abbiamo individuato i valori mancanti, dobbiamo decidere come gestirli. Due approcci semplici e comuni:

1. **Interpolazione lineare**: pandas "collega" il valore prima e dopo il buco con una retta, stimando i valori intermedi.  
    È utile quando credi che il fenomeno cambi gradualmente (es. temperatura che varia lentamente).

2. **Forward fill (riempimento in avanti)**: copia l'ultimo valore valido nei buchi successivi.  
    È utile quando pensi che il valore resti stabile per un po' (es. status di un dispositivo).

**Interpolazione lineare**  
Se alle 10:00 hai 200 kW e alle 11:00 hai 220 kW, ma manca il valore delle 10:30, l'interpolazione stima 210 kW (la metà).  
Comando: `df.interpolate(method="linear")`

**Forward fill**  
Se alle 10:00 hai 200 kW e mancano i valori successivi, forward fill replica 200 kW finché non trova un nuovo valore valido.  
Comando: `df.ffill()` (oppure `df.fillna(method='ffill')`)


</div>

In [ ]:
# lavoriamo SOLO su power_kw
s = df_qh["power_kw"]

t0 = missing_timestamps[0]
# finestra: da t0-45min a t0+45min (slice temporale corretta)
idx = slice(
    t0 - pd.Timedelta(minutes=45),
    t0 + pd.Timedelta(minutes=45),
)

# imputazioni (una colonna ciascuna)
out = pd.DataFrame({
    "original": s.loc[idx],
    "time": s.interpolate(method="time").loc[idx],
    "linear": s.interpolate(method="linear").loc[idx],
    "ffill": s.ffill().loc[idx],
    "bfill": s.bfill().loc[idx],
})

out


I metodi time e linear stimano il valore mancante interpolando tra i due punti vicini.
Con frequenza regolare, i due metodi danno lo stesso risultato (qui 271.25).

In [ ]:
# visualizzazione con Plotly delle imputazioni
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x=out.index, y=out["time"], mode="markers+lines", name="interpolate time"))
fig.add_trace(go.Scatter(x=out.index, y=out["linear"], mode="markers+lines", name="interpolate linear"))
fig.add_trace(go.Scatter(x=out.index, y=out["ffill"], mode="markers+lines", name="ffill"))
fig.add_trace(go.Scatter(x=out.index, y=out["bfill"], mode="markers+lines", name="bfill"))
fig.update_layout(title="Imputazioni per missing value", height=400, width=700)
fig.show()

<div style="background-color:#f5f5f5; padding:12px 14px; border-left:4px solid #5cb85c; border-radius:2px;">

**Esercizio**

Visualizza la serie completa con imputazione tramite interpolazione lineare

</div>

In [ ]:
# usa interpolate su df_qh
imputed_df = ...

# visualizzazione con Plotly (usa px.line)
fig = ...(imputed_df, x=..., y=..., title=...)
fig.show()


<a id="resampling"></a>
## Resampling

Il resampling cambia granularità temporale: da quartorario a orario, giornaliero, mensile…  
È un’operazione di aggregazione guidata dal tempo (non dalla posizione delle righe).

Documentazione:
- `resample`: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.resample.html
- tutorial ufficiale: https://pandas.pydata.org/docs/getting_started/intro_tutorials/09_timeseries.html


In [ ]:
# interpolazione lineare
df_qh = df_qh.interpolate(method="linear")

In [ ]:
hourly = df_qh.resample("h").mean(numeric_only=True) # numeric_only per evitare warning
daily = df_qh.resample("D").mean(numeric_only=True)

hourly.head()

In [ ]:
daily

<div style="background-color:#f5f5f5; padding:12px 14px; border-left:4px solid #5cb85c; border-radius:2px;">

**Esercizio**

Crea un resampling **orario** calcolando la `sum` di `power_kw` (non la media). Poi confronta visivamente le prime righe con `hourly` (che usa la media).

</div>

In [ ]:
hourly_sum = df_qh...
pd.DataFrame({"mean": hourly["power_kw"], "sum": hourly_sum}).head()

<a id="shift-rolling"></a>
## Shift e rolling

Due operazioni semplici e molto usate nella gestione di time series:

- `shift(k)`: sposta i valori di `k` step nel tempo (crea “lag”).
- `rolling(w)`: calcola statistiche su una finestra mobile di ampiezza `w`.

Entrambe possono introdurre `NaN` in testa (perché mancano i valori “precedenti”).

Documentazione:
- `shift`: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.shift.html
- `rolling`: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rolling.html


In [ ]:
df_sr = df_qh.copy()

# lag di 1 ora: con frequenza 15min corrisponde a 4 step
df_sr["power_lag_4"] = df_sr["power_kw"].shift(4)

# media mobile su 2 ore: 8 step
df_sr["power_ma_8"] = df_sr["power_kw"].rolling(8).mean()

df_sr[["power_kw", "power_lag_4", "power_ma_8"]].head(12)

In [ ]:
# visualizzazione con Plotly dello shift di 1 ora (4 step) contro power_kw
fig = px.line(df_sr, x=df_sr.index, y=["power_kw", "power_lag_4"], title="Shift di 1 ora (4 step)")
fig.show()

In [ ]:
df_sr[["power_kw", "power_lag_4"]].head()

In [ ]:
# visualizzazione con Plotly della media mobile su una finestra di 1 ora (4 step) contro power_kw
fig = px.line(df_sr, x=df_sr.index, y=["power_kw", "power_ma_8"], title="Media mobile su finestra di 1 ora (4 step)")

fig.show()

<div style="background-color:#f5f5f5; padding:12px 14px; border-left:4px solid #5cb85c; border-radius:2px;">

**Esercizio**

Gioca cambiando la dimensione della finestra e vedendo il risultato

</div>

<a id="differenze"></a>
## Differenze tra date

Le differenze tra timestamp producono `Timedelta` (durate).  
Servono sia per capire la regolarità della serie sia per misurare intervalli (es. “quante ore copre il dataset?”).

Documentazione:
- `Timedelta`: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Timedelta.html
- `Series.diff`: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.diff.html


In [ ]:
# differenze tra timestamp consecutivi
delta_consecutivi = df_qh.index.to_series().diff()

delta_consecutivi.head(10)

In [ ]:
# intervallo totale coperto
delta_totale = df_qh.index.max() - df_qh.index.min()
delta_totale

In [ ]:
# durata in ore (float)
delta_totale.total_seconds() / 3600

<a id="timezone"></a>
## Fuso orario e ora legale

Un timestamp può essere:
- **naive**: senza fuso orario (solo “data e ora”)
- **timezone-aware**: con fuso orario (es. `Europe/Rome`)

Per dati utility è comune ricevere orari locali “naive” e doverli interpretare correttamente.  
L’ora legale (DST) è la trappola tipica: alcune ore non esistono (primavera) o si ripetono (autunno). 

Documentazione:
- `tz_localize` / `tz_convert`: https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#time-zone-handling
- parametri `ambiguous` e `nonexistent`: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DatetimeIndex.tz_localize.html


<div style="background-color:#f5f5f5; padding:12px 14px; border-left:4px solid #d9534f; border-radius:2px;">

**Attenzione**

Localizzare un indice che attraversa un cambio ora legale può generare timestamp ambigui o inesistenti. 
In quei casi servono i parametri `ambiguous=` e `nonexistent=`. 

Se i dati arrivano già in UTC, spesso è meglio mantenerli in UTC e convertire solo in visualizzazione.

</div>

In [ ]:
df_tz = df_qh.copy()

# Interpretiamo i timestamp come orari locali italiani
df_tz.index = df_tz.index.tz_localize("Europe/Rome")

# Conversione a UTC (utile per sistemi e confronti)
df_utc = df_tz.tz_convert("UTC")

df_tz.index[:3], df_utc.index[:3]

In [ ]:
df_tz.loc["2025-03-01 06:00":"2025-03-01 12:00"].head()